# Lecture 10: Training, Validation and Testing Sets, Cross-validation


## Recap: Linear least squares:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

NUM_POINTS = 10
THETA_0 = 1
THETA_1 = 5

# generate 10 random values of x
x = np.random.random(NUM_POINTS)
# apply linear function, then plot
y = THETA_0 + THETA_1*x
plt.scatter(x, y)

In [ ]:
# add iid noise N(0, 1^2) to y values to form
# noisy observations 
SIGMA_W = 1
w = np.random.normal(0, SIGMA_W, NUM_POINTS)
y_obs = y + w

# plot noisy data
plt.figure()
plt.scatter(x, y_obs)

In [ ]:
# basis functions:
# phi_0(x) = 1
# phi_1(x) = x

phi_0 = np.ones(x.shape)
phi_1 = x

# Design matrix, phi as columns
Phi = np.array([phi_0, phi_1])

# This shows we have the phi as rows
# we don't want that.
print(Phi.shape)

# Take transpose to get phi as columns
Phi = Phi.transpose()
print(Phi.shape)

In [ ]:
# Moore-Penrose pseudoinverse
A = np.matmul(Phi.transpose(), Phi)
A_inv = np.linalg.inv(A)
Phi_inv = np.matmul(A_inv,Phi.transpose())
theta_ls = np.matmul(Phi_inv, y_obs)
print(theta_ls)

In [ ]:
# In reality, calling np.linalg.inv isn't great. 
# It's numerically unstable, and inefficient. We really 
# want to solve the normal equations, and there's 
# better ways to do this, all built into the np.linalg.solve
# function:

A = np.matmul(Phi.transpose(), Phi)
b = np.matmul(Phi.transpose(), y_obs)
theta_ls = np.linalg.solve(A, b)
print(theta_ls)

In [ ]:
# In practice, you should probably use the least squares
# solver that comes with numpy. You can call it with 
# np.linalg.lstsq:

res = np.linalg.lstsq(Phi, y_obs, rcond=None)
theta_ls = res[0]
print(theta_ls)

In [ ]:
# Let's plot the true function, the least-squares 
# fit and the observed data points all in the same plot

# the set of x we will plot the true and fitted model against
x_plot = np.linspace(0, 1, 5)
# the y-values of the true model
y_true = THETA_0 + THETA_1*x_plot
# the y-values of the least squares fitted model
y_model = theta_ls[0] + theta_ls[1]*x_plot

plt.figure()
plt.plot(x_plot, y_true)
plt.plot(x_plot, y_model)
# plot the observations as well
plt.errorbar(x, y_obs, yerr=SIGMA_W, fmt='o')

plt.legend(['True', 'Least Squares Fit', 'Observations'])


### Another example

In [ ]:
# Here we'll load data from a text file
data = np.loadtxt('data.txt')
# We see that there are 2 columns (x, y) and 30 data points
print(data.shape)

In [ ]:
# Look at the data:
x_data = data[:,0]
y_data = data[:,1]
plt.figure()
plt.scatter(x_data, y_data)

In [ ]:
# Here we'll again use the monomial basis, i.e.
# phi_n = x^n for n = 0, 1, ..., NUM_BASIS-1

# Hint: If we're using the monomial basis, the 
# corresponding design matrix Phi has a special
# name called the Vandermonde matrix, and 
# numpy has a special function to construct this matrix
# called np.vander

# fix the number of basis elements to use
NUM_BASIS = 4
# Form the design matrix, aka Vandermonde matrix
Phi = np.vander(x_data, NUM_BASIS, increasing = True)
# Solve for the least squares solution
theta_ls = np.linalg.lstsq(Phi, y_data, rcond=None)[0]
print(Phi.shape)
print(theta_ls)

In [ ]:
# Plot the fitted linear model with data

# Pick a bunch of x values to evaluate the model
x_plot = np.linspace(0, 1, 100)

# Evaluate each basis element at every point 
Phi_plot = np.vander(x_plot, NUM_BASIS, increasing = True)

# Evaluate the model y = Phi*theta
y_plot = np.matmul(Phi_plot, theta_ls)

# Plot the data points and the model fit
plt.figure()
plt.scatter(x_data, y_data)
plt.plot(x_plot, y_plot, color='r')

In [ ]:
# Calculate residual with respect to data
Phi_model = np.vander(x_data, NUM_BASIS, increasing = True)
y_model = np.matmul(Phi_model, theta_ls)
r = y_model - y_data
print("Norm of residual = " + str(np.linalg.norm(r)))

## Training, Vaidation and Testing Sets

In [ ]:
training_set = data[0:20, :]
x_training = training_set[:, 0]
y_training = training_set[:, 1]

validation_set = data[20:24, :]
x_validation = validation_set[:, 0]
y_validation = validation_set[:, 1]

testing_set = data[24:30, :]
x_testing = testing_set[:, 0]
y_testing = testing_set[:, 1]

In [ ]:
# We'll consider 9 models. Each model will use 
# the monomial basis elements {phi_n = x^n}
# The only difference between models is how many
# of these elements to include. 

models_N = [1, 2, 3, 4, 5, 6, 7, 8, 9]

model_performance = np.zeros(len(models_N))

# loop over each model
for (i,N) in enumerate(models_N):
    # train model N (aka N basis elements) with training set
    Phi = np.vander(x_training, N, increasing = True)
    theta_ls = np.linalg.lstsq(Phi, y_training, rcond=None)[0]
    
    # calculate performance with validation set
    Phi_model = np.vander(x_validation, N, increasing = True)
    y_model = np.matmul(Phi_model, theta_ls)
    
    # compare model prediction with validation data
    r = y_model - y_validation
    
    # save performance, here we'll use MSE
    model_performance[i] = np.mean(r**2)

In [ ]:
# Plot each model's performance
plt.figure()
plt.plot(models_N, model_performance)

# Pick best performing model
i_best = np.argmin(model_performance)
N_best = models_N[i_best]
print("Best model: N = " + str(N_best))

In [ ]:
# retrain best model with training set
Phi = np.vander(x_training, N_best, increasing = True)
theta_ls = np.linalg.lstsq(Phi, y_training, rcond=None)[0]
      
# Now compare the final performance of the best model against testing set
Phi_model = np.vander(x_testing, N_best, increasing = True)
y_model = np.matmul(Phi_model, theta_ls)
r = y_model - y_testing

print("MSE for testing set: " + str(np.mean(r**2)))

### Cross-validation

In [ ]:
# k fold cross-validation
training_set = data[0:24, :]

testing_set = data[24:30, :]
x_testing = testing_set[:, 0]
y_testing = testing_set[:, 1]

# pick number of subsets of training sets
# here we'll pick 3 subsets, each with 8 data points
k = 3

# We'll define the 3 sets as
# subset 0 = data points 0, 3, 6,  9, 12, 15, 18, 21 (mod 4 = 0)
# subset 1 = data points 1, 4, 7, 10, 13, 16, 19, 22 (mod 4 = 1)
# subset 2 = data points 2, 5, 8, 11, 14, 17, 20, 23 (mod 4 = 2)
I = np.arange(24)

# loop over models
for (i,N) in enumerate(models_N):

    # perform k validations and save performance for each validation    
    model_performance_j = np.zeros(k)
    for j in range(k):
        # define j-th validation and training sets
        # using boolean indexing
        validation_set_j = training_set[(I % k) == j, :]
        training_set_j = training_set[(I % k) != j, :]
        
        # pick out input and output vectors from training set
        x_training_j = training_set_j[:, 0]
        y_training_j = training_set_j[:, 1]
        
        # pick out input and output vectors from validation set
        x_validation_j = validation_set_j[:, 0]
        y_validation_j = validation_set_j[:, 1]
        
        # train with training set
        Phi = np.vander(x_training_j, N, increasing = True)
        theta_ls = np.linalg.lstsq(Phi, y_training_j, rcond=None)[0]
        
        # calculate residual with validation set
        Phi_model = np.vander(x_validation_j, N, increasing = True)
        y_model = np.matmul(Phi_model, theta_ls)
        r = y_model - y_validation_j
        
        # save performance for the j-th validation set
        model_performance_j[j] = np.mean(r**2)
        
    # Overall model performance for the i-th model is 
    # just the average over all validation sets
    model_performance[i] = np.mean(model_performance_j)
    
# At the end of this loop, model_performance will hold 
# the validation performance for each model

In [ ]:
# Plot each model's performance after cross validation
plt.figure()
plt.plot(models_N, model_performance)

# Pick best performing model
i_best = np.argmin(model_performance)
N_best = models_N[i_best]
print("Best model: N = " + str(N_best))

In [ ]:
# retrain the best model with training set
Phi = np.vander(x_training, N_best, increasing = True)
theta_ls = np.linalg.lstsq(Phi, y_training, rcond=None)[0]
      
# Now compare the final performance of the best model against testing set
Phi_model = np.vander(x_testing, N_best, increasing = True)
y_model = np.matmul(Phi_model, theta_ls)
r = y_model - y_testing

print("MSE for testing set: " + str(np.mean(r**2)))